### Flask 서버 구축

In [1]:
import csv

with open('X_train1.csv', 'r', encoding='utf-8-sig') as f: 
  rdr = csv.reader(f) 
  test = list(rdr)[0]

test = list(test)
for idx, i in enumerate(test):
  test[idx] = i.strip('][').split(', ')

for i in range(len(test)):
  for j in range(len(test[i])):
    test[i][j] = test[i][j][1:-1]

In [2]:
from flask import Flask # 플라스크 클래스 임포트
from flask import request, redirect, Request, Response
import pickle # 데이터 저장/로딩을 위한 모듈
import pandas as pd
import numpy as np
app = Flask(__name__) # 내장변수 name을 이용해 서버를 구동시키는 객체 생성

from tensorflow.keras.models import load_model
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import cx_Oracle

#한글 지원 방법
import os

# model = keras.models.load_model("model.h5")
loaded_model = load_model('best_model.h5')
okt = Okt()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
tokenizer = Tokenizer(19416)
tokenizer.fit_on_texts(test)
max_len = 30

def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  
  return score

@app.route("/GJDiaryService/predict", methods=["GET",'POST'])
def predict() :
    if request.method == 'POST' :
        display(request.form.get('diary')) #사용자가 사용하고 있는 페이지에거 post 방식으로 값을넘겨준다면 form 태그 안에 값으로 넘겨주고 받는다
        # 넘어온 값을 전처리
        diary = request.form.get('diary')
        pre = round(sentiment_predict(diary)*100,1)
        print(str(pre))
        #flask 에서 예측값(모델 실행후의 결과)를 result.jsp 페이지로 넘긴다 예측값은 url(쿼리스트링 방식 사용)
        
        resp = Response(str(pre), mimetype='text/html')
        # cross 도메인 이슈를 해결하기위해 응답 헤더 추가
        resp.headers['Access-Control-Allow-Origin'] = '*'
        return resp

        # return redirect("http://localhost:8081/Diary/calcul_main.jsp?good="+str(pre))
    else :
        # 넘어온 값을 전처리
        diary = request.form.get('diary')
        pre = round(sentiment_predict(diary),1)
        print(str(pre[0]))
        return "get"

if __name__ == "__main__" : # .py 파일에서 main함수 역할
    app.run(host="localhost", port="5000")

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)


'asdasd'

127.0.0.1 - - [12/May/2022 10:09:56] "POST /GJDiaryService/predict HTTP/1.1" 200 -


42.5


...

127.0.0.1 - - [12/May/2022 11:14:47] "POST /GJDiaryService/predict HTTP/1.1" 200 -


35.8


'오늘도 행복합니다.\r\n점심 뭐먹을 생각에 행복합니다.\r\n당신도 행복하나요?'

127.0.0.1 - - [12/May/2022 11:15:36] "POST /GJDiaryService/predict HTTP/1.1" 200 -


98.4
